# Corruption Matrix Benchmark

In [1]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
#np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Fit SPAM model

### Generate True Model, Full POVM and Inital

In [7]:
def generate_spam_benchmark(n=3, c1=1, c2=1):
    d = 2**n

    init_target = InitialState(d, c=c1)
    povm_target = POVM(d, c=c2)

    spam_target = SPAM(init = init_target,
                       povm = povm_target)
    
    return spam_target


def generate_spam_data(spam_target, N_spam=None, shots=1024):
    n = int(np.log2(spam_target.d))
    inputs_spam, _ = generate_pauliInput_circuits(n)
    N_spam = inputs_spam.shape[0]

    state = tf.repeat(spam_target.init.init[None,:,:], N_spam, axis=0)
    state = apply_unitary(state, inputs_spam)
    targets_spam = measurement(state, povm = spam_target.povm.povm)

    #add noise
    if shots != 0:
        targets_spam = add_shot_noise(targets_spam, shots=shots)
        
    return inputs_spam, targets_spam


def spam_pipeline(n, 
                  c1, 
                  c2,
                  shots,
                  use_corr = False,
                  num_iter = 2000):
    spam_target = generate_spam_benchmark(n=n, c1=c1, c2=c2)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, shots=shots)

    
    if use_corr:
        povm = CorruptionMatrix(d)
    else:
        povm = POVM(d)
        
    spam_model = SPAM(init = InitialState(d),
                      povm = povm,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))

    spam_model.pretrain(num_iter = 500, verbose=False)

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = num_iter,
                     verbose = False,
                    )
    
    return spam_target, spam_model

## Three Qubit

In [8]:
n = 3
d = 2**n
c1 = 0.9
c2 = 0.9


num_reps = 10
shots_list = np.linspace(7, 16, 10)
shots_list = 2**shots_list
shots_list = np.append(shots_list, 0)

init_fid_list = np.zeros((len(shots_list), num_reps))
povm_fid_list = np.zeros((len(shots_list), num_reps))

for i, shots in enumerate(tqdm(shots_list)):   
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                shots = shots,
                                                use_corr=False,
                                                num_iter = 2000,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
#saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_full.data"))
saver([init_fid_list, povm_fid_list], "data/spam_fidelity_3qubit_full_shotnoise.data")

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

step 0: loss = 0.01146220664442931
step 100: loss = 0.005418774133147164
step 200: loss = 0.005263088340160698
step 300: loss = 0.00523393298354888
step 400: loss = 0.0052253624466870275
step 500: loss = 0.00522172406403937
step 600: loss = 0.005220473526391432
step 700: loss = 0.005220082239315998
step 800: loss = 0.005219973795319876
step 900: loss = 0.005219947781969559
step 1000: loss = 0.005219942229535362
step 1100: loss = 0.005219941016726358
step 1200: loss = 0.005219940639927567
step 1300: loss = 0.005219940456493003
step 1400: loss = 0.0052199403541300475
step 1500: loss = 0.005219940299802365
step 1600: loss = 0.005219940273237831
step 1700: loss = 0.005219940261059097
step 1800: loss = 0.005219940255663438
step 1900: loss = 0.005219940253285266


  0%|          | 0/2000 [00:00<?, ?it/s]

step 0: loss = 0.010914444531619903
step 100: loss = 0.005044101796054829
step 200: loss = 0.0048433079248347876
step 300: loss = 0.004811373240352975
step 400: loss = 0.004801141946935673
step 500: loss = 0.004794453252449086
step 600: loss = 0.00479138711869378
step 700: loss = 0.004790610978694308
step 800: loss = 0.004790256751395339
step 900: loss = 0.004790086936760714
step 1000: loss = 0.004790009461867462
step 1100: loss = 0.004789976438378662
step 1200: loss = 0.004789963392565634
step 1300: loss = 0.004789958639500511
step 1400: loss = 0.004789957048317212
step 1500: loss = 0.004789956560505716
step 1600: loss = 0.0047899564240213525
step 1700: loss = 0.004789956389298378
step 1800: loss = 0.004789956381297838
step 1900: loss = 0.004789956379635518


  0%|          | 0/2000 [00:00<?, ?it/s]

step 0: loss = 0.010110247152940288
step 100: loss = 0.004338688138458652
step 200: loss = 0.004199161090977416
step 300: loss = 0.004166413874483931
step 400: loss = 0.004157944817488053
step 500: loss = 0.004154630986824541
step 600: loss = 0.004153168147076816
step 700: loss = 0.004152474581568065
step 800: loss = 0.00415212418404796
step 900: loss = 0.004151943296356147
step 1000: loss = 0.004151849734193161
step 1100: loss = 0.004151801474421031
step 1200: loss = 0.004151776727706553
step 1300: loss = 0.0041517641859551814
step 1400: loss = 0.004151757956149476
step 1500: loss = 0.0041517549510005185
step 1600: loss = 0.0041517535552914086
step 1700: loss = 0.00415175293573543
step 1800: loss = 0.0041517526744474335
step 1900: loss = 0.004151752570269913


  0%|          | 0/2000 [00:00<?, ?it/s]

step 0: loss = 0.009994755677235644
step 100: loss = 0.004360556861564928
step 200: loss = 0.004222997445799836
step 300: loss = 0.004191551861324445
step 400: loss = 0.004182716993100503
step 500: loss = 0.0041798914065940985
step 600: loss = 0.004178503016382505
step 700: loss = 0.004177699607710576
step 800: loss = 0.00417720575725375
step 900: loss = 0.004176888907754528
step 1000: loss = 0.0041766838157024765
step 1100: loss = 0.0041765524629175385
step 1200: loss = 0.004176470117113083
step 1300: loss = 0.004176419931339117
step 1400: loss = 0.004176390325808803
step 1500: loss = 0.004176373444198176
step 1600: loss = 0.0041763640990829624
step 1700: loss = 0.004176358999688775
step 1800: loss = 0.00417635619110659
step 1900: loss = 0.0041763546393945895


  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
n = 3
d = 2**n
c1 = 0.9
c2 = 0.9


num_reps = 10
shots_list = np.linspace(7, 16, 10)
shots_list = 2**shots_list
shots_list = np.append(shots_list, 0)

init_fid_list = np.zeros((len(shots_list), num_reps))
povm_fid_list = np.zeros((len(shots_list), num_reps))

for i, shots in enumerate(tqdm(shots_list)):   
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                shots = shots,
                                                use_corr=True,
                                                num_iter = 2000,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
#saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_full.data"))
saver([init_fid_list, povm_fid_list], "data/spam_fidelity_3qubit_corr_shotnoise.data")

  0%|          | 0/11 [00:00<?, ?it/s]

## Four Qubit

In [ ]:
n = 4
d = 2**n
c1 = 0.9
c2 = 0.9


num_reps = 10
shots_list = np.linspace(7, 16, 10)
shots_list = 2**shots_list
shots_list = np.append(shots_list, 0)

init_fid_list = np.zeros((len(shots_list), num_reps))
povm_fid_list = np.zeros((len(shots_list), num_reps))

for i, shots in enumerate(tqdm(shots_list)):   
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                shots = shots,
                                                use_corr=False,
                                                num_iter = 2000,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
#saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_full.data"))
saver([init_fid_list, povm_fid_list], "data/spam_fidelity_4qubit_full_shotnoise.data")

  0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
n = 4
d = 2**n
c1 = 0.9
c2 = 0.9


num_reps = 10
shots_list = np.linspace(7, 16, 10)
shots_list = 2**shots_list
shots_list = np.append(shots_list, 0)

init_fid_list = np.zeros((len(shots_list), num_reps))
povm_fid_list = np.zeros((len(shots_list), num_reps))

for i, shots in enumerate(tqdm(shots_list)):   
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                shots = shots,
                                                use_corr=True,
                                                num_iter = 2000,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
#saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_full.data"))
saver([init_fid_list, povm_fid_list], "data/spam_fidelity_4qubit_corr_shotnoise.data")

  0%|          | 0/11 [00:00<?, ?it/s]

c:\Users\krist\anaconda3\envs\env_qiskit\lib\site-packages\tensorflow\python\framework\ops.py:1064: ComplexWarning: Casting complex values to real discards the imaginary part
  return float(self._numpy())


FileNotFoundError: [Errno 2] No such file or directory: '../../data\\spam_fidelity_4qubit_corr_shotnoise.data'